<a href="https://colab.research.google.com/github/aliyaaliyal/mesin-learning/blob/main/stacking-LIME-streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sangkarsana/ulasan_pinjol/main/pinjol_raw10rb.csv')
df.head(10)

,Unnamed: 0,content,score,at
0,8585,Aplikasi apaan ni? Dpt rekom kalo aplikasi ini...,1,2020-05-31 13:39:28
1,8035,Saya kasih bintang 4 hampir sempurna karena pr...,4,2020-06-05 11:42:46
2,7223,Ko aneh yaa pencairan blm ada tp udh ada tgl p...,1,2020-06-08 08:30:14
3,9788,"halo adakami, pinjaman saya di setujui dan ter...",2,2020-06-11 19:41:53
4,5018,Cukup bagus tapi tolong untk info penagihan bi...,1,2020-06-15 02:18:19
5,8780,Sdh lunas.. eh gak bisa diajuin lg.. alias di ...,3,2020-06-15 07:52:31
6,9406,"Terimakasih ada kami, pelayanan yg sangat memu...",5,2020-06-17 00:27:13
7,9883,"Hallo, Ada kami saya tidak jadi meminjam di Ad...",2,2020-06-18 00:57:53
8,9803,"Gampang gimana, masa suruh meningkatkan scorin...",1,2020-06-18 12:23:35
9,4973,"proses cepat, tapi untuk ke depannya tenor aga...",2,2020-06-18 12:32:55


In [ ]:
# Fungsi untuk melabeli sentimen berdasarkan skor ulasan
def label_sentiment(score):
    if score >= 4:
        return 'Positif'
    #elif score == 3:
    #    return 'Netral'
    else:
        return 'Negatif'

# Menerapkan fungsi ke kolom 'score' untuk membuat kolom 'sentimen'
df['sentiment'] = df['score'].apply(label_sentiment)

# Menampilkan 5 baris teratas untuk memastikan pelabelan berhasil
df.head(5)

,Unnamed: 0,content,score,at,sentiment
0,8585,Aplikasi apaan ni? Dpt rekom kalo aplikasi ini...,1,2020-05-31 13:39:28,Negatif
1,8035,Saya kasih bintang 4 hampir sempurna karena pr...,4,2020-06-05 11:42:46,Positif
2,7223,Ko aneh yaa pencairan blm ada tp udh ada tgl p...,1,2020-06-08 08:30:14,Negatif
3,9788,"halo adakami, pinjaman saya di setujui dan ter...",2,2020-06-11 19:41:53,Negatif
4,5018,Cukup bagus tapi tolong untk info penagihan bi...,1,2020-06-15 02:18:19,Negatif


In [ ]:
import re # regex library
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [ ]:
from sklearn.model_selection import train_test_split
X = df['content']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, max_features=700, preprocessor=preprocessor, ngram_range=(1,1))
neural_net_pipeline = Pipeline([('vectorizer', tfidf), ('nn', MLPClassifier(hidden_layer_sizes=(700, 700)))])
neural_net_pipeline.fit(X_train, y_train)

In [ ]:
# Base models
rf = RandomForestClassifier()
nb = MultinomialNB()
svm = SVC(probability=True)  # SVM dengan probability=True untuk stacking

# Stacking classifier dengan Logistic Regression sebagai meta-model
stack = StackingClassifier(
    estimators=[
        ('rf', rf),
        ('nb', nb),
        ('svm', svm)
    ],
    final_estimator=LogisticRegression()
)

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, max_features=700, preprocessor=preprocessor, ngram_range=(1,1))
neural_net_pipeline = Pipeline([
    ('vectorizer', tfidf),
    ('classifier', stack)  # Ganti 'nn' dengan 'classifier' dan gunakan stack
])
neural_net_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(lowercase=False, max_features=700,
                                 preprocessor=<function preprocessor at 0x78f7d982d990>)),
                ('classifier',
                 StackingClassifier(estimators=[('rf',
                                                 RandomForestClassifier()),
                                                ('nb', MultinomialNB()),
                                                ('svm', SVC(probability=True))],
                                    final_estimator=LogisticRegression()))])

In [ ]:
y_pred = neural_net_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print('Accuracy: {} %'.format(100 * accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

     Negatif       0.88      0.93      0.91      1891
     Positif       0.87      0.79      0.83      1109

    accuracy                           0.88      3000
   macro avg       0.88      0.86      0.87      3000
weighted avg       0.88      0.88      0.88      3000

Accuracy: 87.83333333333333 %


In [ ]:
from joblib import dump
dump(neural_net_pipeline, 'star.joblib')

['star.joblib']

In [ ]:
pip install lime -qqqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
%%writefile app.py
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import streamlit as st
from lime.lime_text import LimeTextExplainer
import streamlit.components.v1 as components

st.write("# Klasifikasi Ulasan Pinjol")

message_text = st.text_input("Masukkan teks yang ingin diprediksi")

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

model = joblib.load('star.joblib')

def classify_message(model, message):

	label = model.predict([message])[0]
	spam_prob = model.predict_proba([message])

	return {'label': label, 'Kemungkinan Negatif': spam_prob[0][0], 'Kemungkinan Positif': spam_prob[0][1]}

if message_text != '':

	result = classify_message(model, message_text)

	st.write(result)


	explain_pred = st.button('Explain Predictions')

	if explain_pred:
		with st.spinner('Generating explanations'):
			class_names = ['Negatif', 'Positif']
			explainer = LimeTextExplainer(class_names=class_names)
			exp = explainer.explain_instance(message_text,
				model.predict_proba, num_features=10)
			components.html(exp.as_html(), height=800)

Overwriting app.py


In [ ]:
!pip install streamlit -qqqq

In [ ]:
!pip install pyngrok

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session c09e3c9452328bd


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.123.108.233:8501

npx: installed 22 in 5.094s
your url is: https://blue-ducks-tease.loca.lt
  Stopping...
^C
